# Salford Dataset - First Look

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

%load_ext autoreload
%autoreload 1
%aimport datasets.salford, datasets.salford_raw, datasets.icd10, datasets.ccs
from datasets.salford import SalfordData, SalfordFeatures
from datasets.salford_raw import SalfordTimeseries
from datasets.icd10 import ICD10Table
from datasets.ccs import CCSTable

class Notebook:
    DATA_DIR = 'data/Salford'
    FROM_EXCEL = False

We load the raw dataset. We can extract the data from the original .xlsx files, but this takes ~20 minutes. We prefer to load the pre-extracted dataframe.

In [2]:
def load_salford_from_excel():
    infiles = [f'{Notebook.DATA_DIR}/part1.xlsx', f'{Notebook.DATA_DIR}/part2.xlsx']
    xlsx = [pd.read_excel(_) for _ in infiles]
    df = pd.concat(xlsx)
    df.to_hdf(f'{Notebook.DATA_DIR}/raw.h5', 'table')
    return df

if Notebook.FROM_EXCEL:
    df = load_salford_from_excel()
else:
    df = pd.read_hdf(f'{Notebook.DATA_DIR}/raw.h5', 'table')


We load the raw dataframe into a SalfordData instance. Calling `from_raw` also performs the basic pre-processing steps for us. Calling `augment_derive_all` augments the dataset with some composite features, such as critical care and mortality outcomes or identifying readmissions.

In [3]:
sal = SalfordData.from_raw(df)#.augment_derive_all()

## Data Availability
We inspect the data availability of the timeseries features.

In [4]:
timeseries_availability = (pd.DataFrame([(
        feature.split('_')[0],
        feature.split('_')[1],
        sal[cols[0]].notna().sum(),
        sal[cols[1]].notna().sum(),
        sal[cols[2]].notna().sum(),
        sal[cols[3]].notna().sum()
    ) for feature, cols in SalfordTimeseries.items()],
    columns=['Group', 'Feature', 'Admission', '24H Post-Adm', '24H Pre-Disch', 'Discharge']
).set_index(['Group', 'Feature'])/sal.shape[0]*100).round(2)
timeseries_availability.loc[['NEWS', 'Blood', 'VBG']]

Admission  24H Post-Adm  24H Pre-Disch  Discharge
Group Feature                                                           
NEWS  Score                71.53         46.97          49.64      72.06
      RespiratoryRate      67.41         44.59          47.04      68.15
      BreathingDevice      67.41         44.59          47.04      68.15
      O2Sat                67.41         44.59          47.04      68.15
      Temperature          67.41         44.59          47.04      68.15
      BP                   67.41         44.59          47.04      68.15
      HeartRate            67.41         44.59          47.04      68.15
      AVCPU                67.41         44.59          47.04      68.15
Blood Haemoglobin          79.65         13.17           3.64      39.11
      Urea                 78.57         14.41           4.48      41.03
      Sodium               78.57         15.07           4.77      42.25
      Potassium            69.45         14.62           4.76      42.31
      Creatinine           78.51         14.74           6.08      65.22
      DDimer                5.58          0.00           0.00       0.42
      CRP                  66.12          7.71           2.08      31.15
      Albumin              67.23         10.64           3.10      36.62
      WhiteCount           79.65         13.17           3.64      39.10
VBG   Temperature           6.66          0.05           0.03       1.31
      pCO2                  6.65          0.05           0.03       1.29
      pCO2                  6.65          0.05           0.03       1.29
      PH                    6.65          0.05           0.03       1.29
      PH                    6.65          0.05           0.03       1.29
      O2                    6.64          0.05           0.03       1.29
      O2                    6.64          0.05           0.03       1.29

In [7]:
# print(timeseries_availability.to_latex(escape=False, index=True, multirow=True, formatters=[
#     lambda x: f'${x:.2f}\%$' for _ in range(4)
# ]))

We do the same across all features.

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print((sal.notna().sum(axis=0)/sal.shape[0]*100))#.apply(lambda x: f'${x:.2f}\%$'))

PatientNumber                       100.000000
Female                              100.000000
Age                                 100.000000
Ethnicity                            99.629844
AE_PresentingComplaint               66.599743
AE_MainDiagnosis                     60.826201
AE_Arrival                           68.299822
AE_Departure                         68.299822
AE_Location                          68.082610
AE_PatientGroup                      68.299822
AE_TriageNote                        67.937885
AdmissionDate                       100.000000
DischargeDate                       100.000000
ElectiveAdmission                   100.000000
AdmitMethod                         100.000000
AdmissionSpecialty                  100.000000
DischargeConsultant                 100.000000
DischargeSpecialty                  100.000000
TotalLOS                            100.000000
LOSBand                             100.000000
AdmitWard                            99.825384
AdmitWardLOS 

## ICD-10 Groupings

We fuzzily match the recorded ICD-10 coded diagnoses with the standard lookup table. Some limited losses occur.

In [12]:
icd = ICD10Table.fuzzy_match(sal.Diagnoses)
print(f'Values that we lost due to ICD-10 matching: {sal.Diagnoses.notna().values.sum() - icd.notna().values.sum()} out of {sal.Diagnoses.notna().values.sum()}')

Values that we lost due to ICD-10 matching: 1018 out of 3134818


We may then conver this standardised format into [CCS](https://ibis.health.utah.gov/ibisph-view/pdf/resource/CCS.pdf) diagnosis groups.

In [11]:
ccs = CCSTable.fuzzy_match(icd)
print(f'Values that we lost due to CCS matching: {ccs.notna().values.sum() - icd.notna().values.sum()}')

Values that we lost due to CCS matching: 0


These are then easy to convert to more compact formats, like the HSMR aggregate groups.

In [14]:
# Group from scratch using the SalfordData method
# Use clean_icd10=True to perform the ICD-10 matching before converting to CCS and subsequently HSMR
sal.derive_ccs(return_df=True, clean_icd10=True, grouping='HSMR').sample(5).reset_index(drop=True)

,MainICD10,SecDiag1,SecDiag2,SecDiag3,SecDiag4,SecDiag5,SecDiag6,SecDiag7,SecDiag8,SecDiag9,SecDiag10,SecDiag11,SecDiag12,SecDiag13,SecDiag14,SecDiag15
0,17.0,7.0,7.0,17.0,6.0,7.0,7.0,6.0,5.0,12.0,17.0,17.0,17.0,NaN,NaN,NaN
1,8.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.0,7.0,10.0,7.0,7.0,8.0,7.0,3.0,9.0,3.0,17.0,17.0,17.0,17.0,17.0,NaN
3,6.0,6.0,7.0,7.0,17.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.0,17.0,8.0,8.0,8.0,7.0,9.0,7.0,6.0,17.0,17.0,17.0,NaN,NaN,NaN,NaN
